# Federated Learning across Nodes Training a CNN model

New script to run a CNN-based federated system, where we can vary the inner and outer learning rates

In [1]:
#Imports
import os    
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
from sklearn import preprocessing
from sklearn import metrics
from keras.datasets import mnist
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import random
import csv
import timeit
from timeit import default_timer as timer

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck']
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

Functions to deal with splitting of data or creating models. Basically non-interesting functions

In [2]:
def splittingData(x_train,y_train,x_test,y_test,noClients):
    trainLen = len(x_train)
    clientSize = trainLen/noClients
    
    x_train_clients = []
    y_train_clients = []
    x_test_clients = []
    y_test_clients = []
    
    x_train_splits = np.array_split(x_train,noClients)
    y_train_splits = np.array_split(y_train,noClients)
    x_test_splits = np.array_split(x_test,noClients)
    y_test_splits = np.array_split(y_test,noClients)
    
    for i in range(noClients):
        x_train_clients.append(x_train_splits[i])
        y_train_clients.append(y_train_splits[i])
        x_test_clients.append(x_test_splits[i])
        y_test_clients.append(y_test_splits[i])
    
    return x_train_clients, y_train_clients, x_test_clients, y_test_clients

def createModel(iLr):
    #optimizer = tf.keras.optimizers.Adam(learning_rate=iLr)
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10))
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
    return model

def createZeroWeightCNN(iLr):
    layer1 = [np.zeros(shape = (3,3,3,32), dtype = 'float32'), np.zeros(shape = 32, dtype = 'float32')]
    layer2 = [np.zeros(shape = (3,3,32,64), dtype = 'float32'), np.zeros(shape = 64, dtype = 'float32')]
    layer3 = [np.zeros(shape = (10816,64), dtype = 'float32'), np.zeros(shape = (64), dtype = 'float32')]
    layer4 = [np.zeros(shape = (64,10), dtype = 'float32'), np.zeros(shape = (10), dtype = 'float32')]

    testModel = models.Sequential()
    testModel.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    testModel.add(layers.MaxPooling2D((2, 2)))
    testModel.add(layers.Conv2D(64, (3, 3), activation='relu'))
    testModel.add(layers.Flatten())
    testModel.add(layers.Dense(64, activation='relu'))
    testModel.add(layers.Dense(10))
    optimizer = tf.keras.optimizers.Adam(learning_rate=iLr)
    testModel.layers[0].set_weights(layer1)
    testModel.layers[2].set_weights(layer2)
    testModel.layers[4].set_weights(layer3)
    testModel.layers[5].set_weights(layer4)
    testModel.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
    return testModel



Plotting Functions

In [3]:
def plottingDataSets(data, datat):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    numClasses = np.arange(1,len(class_names)+1)
    for i in range(nClients):
    #for i in range(nClients):
        ns = np.zeros(len(class_names))
        nst = np.zeros(len(class_names))
        for j in range(len(data[0])):
            ns[data[i][j][0]] += 1
        for j in range(len(datat[0])):
            nst[datat[i][j][0]] += 1

        ax1.bar(numClasses,ns,alpha=0.5,label="Training set {0}".format(i)) 
        ax2.bar(numClasses,nst,alpha=0.5,label="Testing set {0}".format(i)) 

    plt.suptitle("The Count of Each Class", fontsize = 'x-large') 
    ax1.set_title("Clients training Dataset") 
    ax2.set_title("Clients testing Dataset")
    ax1.set(ylabel = "Count", xlabel = "Class")
    ax2.set(ylabel = "Count", xlabel = "Class")
    ax2.legend(bbox_to_anchor=(1.1, 1.05))
    ax2.yaxis.set_label_coords(-0.1, 0.5)
    ax1.set_xticks(numClasses)
    ax2.set_xticks(numClasses)
    plt.show()
    
def plottingFreq(data,datat, class_):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    numClients = np.arange(1,nClients+1)
    ncls = np.zeros(nClients)
    nclst = np.zeros(nClients)
    for i in range(nClients):
    #for i in range(nClients):
        for j in range(len(data[0])):
            if (data[i][j][0] == class_):
                ncls[i] += 1
        for j in range(len(datat[0])):
            if (datat[i][j][0] == class_):
                nclst[i] += 1

    ax1.bar(numClients,ncls,alpha=1,label="Training set {0}".format(i)) 
    ax2.bar(numClients,nclst,alpha=1,label="Testing set {0}".format(i)) 

    fig.suptitle("The Count of the Class "+ str(class_) + " for each Client", fontsize='x-large')
    ax1.set_title("Clients training Dataset") 
    ax2.set_title("Clients testing Dataset")
    ax1.set(ylabel = "Count", xlabel = "Client Number")
    ax2.set(ylabel = "Count", xlabel = "Client Number")
    ax1.set_xticks(numClients)
    ax2.set_xticks(numClients)
    ax2.yaxis.set_label_coords(-0.1, 0.5)
    #ax.legend(bbox_to_anchor=(1.1, 1.05))
    plt.show()
def plotting(data,datat):
    plt.rcParams['figure.figsize'] = [8,4]
    plottingDataSets(y_train_clients,y_test_clients)
    c = np.random.choice(clients)
    plottingFreq(y_train_clients,y_test_clients, c)

Federated Functions, where fedAvg now uses a global learning rate by the following equation:
$$
  G^t = G^{t-1} + \dfrac{\eta}{m}\sum_{i=1}^{m}(P_i^t - G^{t-1}).
  \label{hi}
$$
Our loss is taken as the difference between our local model and the global model

In [65]:
def McMahanFedAvg(models_,fedModel,numClients,l):
    #setup an array to hold the weights and a null array to hold the summation of weights in the federated system
    layersNums = [0,2,4,5]    
    fedModelWeights = []
    mmSum = []
    for i in layersNums:
        fedModelWeights.append(fedModel.layers[i].weights)
        mmSum.append(fedModel.layers[i].weights)
    
    for i in range(len(layersNums)):
        for j in range(len(models_[0].layers[layersNums[i]].weights)):
            mmSum[i][j] = mmSum[i][j]*0
    
    #Perform the summation
    for n in range(0,numClients):
        count = 0
        for i in layersNums:
            for j in range(len(models_[0].layers[i].weights)):
                #mmSum contains the summation of the gradients as seen in the equation outlined in the markdown above
                mmSum[count][j] = models_[n].layers[i].weights[j] - fedModelWeights[count][j]
            count += 1
    
    #find the new global models parameters
    count = 0
    for i in layersNums:
        for j in range(len(models_[0].layers[i].weights)):
            mmSum[count][j] = (l/numClients)*mmSum[count][j]
            fedModelWeights[count][j] = fedModelWeights[count][j] + mmSum[count][j]
        count += 1
    
    #Update the global model
    count = 0
    for i in layersNums:
        fedModel.layers[i].set_weights(fedModelWeights[count])
        count += 1
    return fedModel


def runFed(numClients, noEpochs,nUpdates,bSize, writer, oLr,iLr, v):
    #create local and global models
    row = []
    fedModel = createModel(iLr)
    models = []
    for i in range(numClients):
        model = createModel(iLr)
        models.append(model)
    
    batches = np.arange(bSize)    
    #training the CNN
    for u in range(nUpdates):
        start = timer()
        clients = np.arange(nClients)
        #Send the current model to all clients
        for i in range(nClients):
            models[i] = fedModel
        #For a batch of clients, train the model
        for i in range(bSize):
            b = np.random.choice(clients)
            #Delete the selected client in the batch from being selected again in next iteration
            clients = np.setdiff1d(clients,b)
            models[b].fit(x_train_clients[b], y_train_clients[b], epochs=noEpochs, verbose=v)
        #Performing our aggregation
        fedModel = McMahanFedAvg(models,fedModel,numClients,oLr)
        #testing the CNN
        testModelsFed(u,fedModel,numClients,writer,start,numClients,v,0)

    return fedModel, models

def runFedPoisoned(numClients, noEpochs,nUpdates,bSize, writer, oLr,iLr, v):
    #create local and global models
    row = []
    fedModel = createModel(iLr)
    models = []
    for i in range(numClients):
        model = createModel(iLr)
        models.append(model)
    
    batches = np.arange(bSize)    
    #training the CNN
    for u in range(nUpdates):
        start = timer()
        clients = np.arange(nClients-1)
        #Send the current model to all clients
        for i in range(nClients):
            models[i] = fedModel
        #For a batch of clients, train the model
        for i in range(bSize):
            if (u == nUpdates-2):
                print("label-Flipped client selected")
                b = clients[-1]
            else:                
                b = np.random.choice(clients)
            #Delete the selected client in the batch from being selected again in next iteration
            clients = np.setdiff1d(clients,b)
            models[b].fit(x_train_clients[b], y_train_clients[b], epochs=noEpochs, verbose=v)
        #Performing our aggregation
        fedModel = McMahanFedAvg(models,fedModel,numClients,oLr)
        #testing the CNN
        testModelsFed(u,fedModel,numClients,writer,start,numClients,v,0)

    return fedModel, models
#Testing the model
def testModelsFed(u,model,numClients,writer,start,NC,v,print_):
    fedAcc = 0
    row = [numClients, u+1,0,0]
    for i in range (NC):
        row.append(0)
    for i in range(numClients):
        #tesing takes place here
        loss, acc = model.evaluate(x_test_clients[i],  y_test_clients[i], verbose=v)
        fedAcc += acc/numClients
        row[4+i] = acc
    print("Accuracy at update {0} is = {1}".format(u,fedAcc))
    end = timer()
    row[2] = fedAcc
    row[3] = end-start
    #writing to .txt file
    if (print_ == 1):
        writer.writerow(row)

Now lets build a model that takes into account everything we know so far, the learning rate(s), batch size, the number of clients, the number of epochs per training round and the number of training rounds

In [66]:
#Setting Seeds
os.environ['PYTHONHASHSEED']=str(2)
tf.random.set_seed(2)
np.random.seed(2)
random.seed(2)
#Variables to alter
nClients = 10
nUpdates = 5
E = 10
outerLr = 0.01
innerLr = 0.001
C = 0.5
B = int(np.round(nClients*C))
#My own variables to print or to control verbose
verbose = 0
print_ = 0

In [67]:
#Opening a file to print into
f = open('./FCNNResults.csv', 'w', newline = '')
header = ['NumClients', 'NumUpdates','FTA','Time']
clients = []
for i in range(nClients):
    header.append("LAN{0}".format(i))
    clients.append(i+1)
writer = csv.writer(f)
if (print_ == 1):
    writer.writerow(header)

Lets play around with our data

In [68]:
def nonIID(x_train,y_train,x_test,y_test,noClients):
    trainLen = len(x_train)
    clientSize = trainLen/noClients
    nClasses = np.arange(len(class_names))
    
    x_train_clients = []
    y_train_clients = []
    x_test_clients = []
    y_test_clients = []
    x_train_new = []
    y_train_new = []
    for i in range(len(class_names)):
        for j in range(len(train_images)):
            if (y_train[j] == i):
                x_train_new.append(x_train[j])
                y_train_new.append(y_train[j])
                
    x_train_splits = np.array_split(x_train_new,noClients)
    y_train_splits = np.array_split(y_train_new,noClients)
    x_test_splits = np.array_split(x_test,noClients)
    y_test_splits = np.array_split(y_test,noClients)
    
    for i in range(noClients):
        x_train_clients.append(x_train_splits[i])
        y_train_clients.append(y_train_splits[i])
        x_test_clients.append(x_test_splits[i])
        y_test_clients.append(y_test_splits[i])
    
    return x_train_clients, y_train_clients, x_test_clients, y_test_clients

def poisoningSplit(x_train,y_train,x_test,y_test,noClients):
    trainLen = len(x_train)
    clientSize = trainLen/noClients
    
    x_train_clients = []
    y_train_clients = []
    x_test_clients = []
    y_test_clients = []
    
    x_train_splits = np.array_split(x_train,noClients)
    y_train_splits = np.array_split(y_train,noClients)
    x_test_splits = np.array_split(x_test,noClients)
    y_test_splits = np.array_split(y_test,noClients)
    
    for i in range(noClients):
        x_train_clients.append(x_train_splits[i])
        y_train_clients.append(y_train_splits[i])
        x_test_clients.append(x_test_splits[i])
        y_test_clients.append(y_test_splits[i])
    classes = [0,9]
    for i in range(len(y_train_clients[-1])):
        if (y_train_clients[-1][i] == classes[0]):
            y_train_clients[-1][i] = classes[1]
        elif (y_train_clients[-1][i] == classes[1]):
            y_train_clients[-1][i] = classes[0]
    return x_train_clients, y_train_clients, x_test_clients, y_test_clients

In [63]:
x_train_clients, y_train_clients, x_test_clients, y_test_clients = splittingData(train_images, train_labels, test_images, test_labels, nClients)
print("Running a federated system with {0} clients, using {1} epochs per update, over {2} updates with a batch size of {3}".format(nClients,E, nUpdates,B))
fedModel, finalLocalModels = runFed(nClients, E, nUpdates,B, writer,outerLr,innerLr,verbose)

Running a federated system with 10 clients, using 10 epochs per update, over 5 updates with a batch size of 5
Accuracy at update 0 is = 0.5908000051975251
Accuracy at update 1 is = 0.6004999935626983
Accuracy at update 2 is = 0.6069000005722046
Accuracy at update 3 is = 0.6118000090122222
Accuracy at update 4 is = 0.6051999926567078


In [69]:
x_train_clients, y_train_clients, x_test_clients, y_test_clients = poisoningSplit(train_images, train_labels, test_images, test_labels, nClients)
print("Running a federated system with {0} clients, using {1} epochs per update, over {2} updates with a batch size of {3}".format(nClients,E, nUpdates,B))
fedModel, finalLocalModels = runFedPoisoned(nClients, E, nUpdates,B, writer,outerLr,innerLr,verbose)

Running a federated system with 10 clients, using 10 epochs per update, over 5 updates with a batch size of 5
Accuracy at update 0 is = 0.5928999960422515
Accuracy at update 1 is = 0.5961000025272369
Accuracy at update 2 is = 0.6080999970436096
label-Flipped client selected
label-Flipped client selected
label-Flipped client selected
label-Flipped client selected
label-Flipped client selected
Accuracy at update 3 is = 0.6074000060558319
Accuracy at update 4 is = 0.6026000022888183


In [228]:
x_train_clients, y_train_clients, x_test_clients, y_test_clients = nonIID(train_images, train_labels, test_images, test_labels, nClients)
print("Running a federated system with {0} clients, using {1} epochs per update, over {2} updates with a batch size of {3}".format(nClients,E, nUpdates,B))
fedModel, finalLocalModels = runFed(nClients, E, nUpdates,B, writer,outerLr,innerLr,verbose)

Running a federated system with 100 clients, using 10 epochs per update, over 5 updates with a batch size of 10
Accuracy at update 0 is = 0.0999999998137355
Accuracy at update 1 is = 0.10000000018626456
Accuracy at update 2 is = 0.09999999983236194
Accuracy at update 3 is = 0.10000000007450585
Accuracy at update 4 is = 0.10000000022351746
